In [1]:
import numpy as np

# Confidence Intervals

A common task is to use the result of our point estimator together with its mean square error to compute an interval in which we expect to often see the parameter of interest. Again we will assign a level of confidence to this statement in the form of a probability, and our interpretation of that is that it is a statement about the repeatability of the result. 

## Example

We are going to sample an exponential random variable $Y$ once and use that value to estimate the mean $\theta$ of the distribution.  We will then use the value of $Y$ to construct an interval in which we are 99% confident we will find $\theta$.

The PDF for the distribution is therefore:

$$ f(y) = \left\{ \begin{matrix} \frac{1}{\theta} e^{-y/\theta} & y \geq 0 \\ 0 & \mbox{otherwise} \end{matrix} \right. $$

The standard exponential (the exponential distribution with mean 1) is found from $Y$ by scaling it by $\theta$:  

$$ U = Y / \theta $$ 

will have the distribution 

$$ f_U(u) = \left\{ \begin{matrix} e^{-u} & u \geq 0 \\ 0 & \mbox{otherwise} \end{matrix} \right. $$

Which we need because this is the exponential distribution encoded in scipy.stats.expon.

We need to find two numbers $a$ and $b$ such that 

$$ P(a < U < b) = 0.99 $$

Usually we would do this so that the tails are symmetric:

$$ P( U < a) = 0.005 \quad \mbox{and} \quad P(U > b) = 0.005 $$

However it is worth noting that there are cases where we might want to have this be asymmetric such as by taking a=0 and choosing the b tail to have size 0.01.

In [3]:
# let's bring in the relevant distribution

from scipy.stats import expon

In [5]:
# We can find a by using the ppf, the inverse cdf:

a = expon.ppf(0.005)
a

0.005012541823544282

In [7]:
# Check with the CDF that this is the left tail at size 0.005

expon.cdf(a)

0.005

In [8]:
# To find b we need to take 1 - 0.005 as our inverse for the cdf:

b = expon.ppf(1-0.005)
b

5.298317366548035

In [9]:
# Check that the density between a and b is the 0.99 we want

expon.cdf(b) - expon.cdf(a)

0.99

Note why this example is not symmetric:  We are using a distribution that is skewed left and only supported from $[0, \infty)$.

Okay so we have that:

$$ 0.99 = P(a < \frac{Y}{\theta} < b ) $$ 

and therefore solving for $\theta$ we get:

$$ 0.99 = P( \frac{Y}{b} < \theta < \frac{Y}{a} ) $$

Noting that we had to flip the inequalities when we took the reciprical. 

#### Suppose $Y= 5$

Then our conclusion is that there is a 99% probability that the mean of the exponential distribution we just sampled is in the interval:

In [10]:
(5/b, 5/a)

(0.9436958290887744, 997.4979114417815)

#### Why so big?

This interval is huge!  Why is it so big?

Again we see these competing tensions:  adjusting our confidence will adjust the size of the region we end up with; and sampling our variable enough times will change the distribution for our point estimate.

## Using a Point Estimate from a Sample

Consider the following sample from the exponential distribution with mean $\theta$.  Let's pretend we do not know what $\theta$ is:

In [43]:
n = 20
sample = expon.rvs(size=n)*10
sample

array([19.14507929,  0.67014224,  6.99290417,  9.32904158,  1.73986975,
        3.31643215,  1.66717348, 26.28050942,  2.27332064,  0.70717216,
       14.06590027, 23.07286012, 13.13140282, 10.54802346,  0.83651637,
        7.30800148,  4.54687187,  5.21351631,  8.12527497,  8.44436276])

In [44]:
# Our point estimate for the mean would be the sample mean:

Ybar = np.mean(sample)
Ybar

8.37071876607573

Which we expect to be normally distributed with mean $\theta$ and variance $\sigma^2 / \sqrt{n}$.  The problem is that we do not know $\sigma^2$.  We can use the sample variance $s^2$ instead, but it means rather than a normal distribution we expect that 

$$ T = \sqrt{n} \frac{\bar{Y} - \theta}{s} $$

will satisfy the Student's T distribution with $n-1$ degrees of freedom.

In [45]:
# Let me introduce you to the numpy.std function for finding standard deviations;
# you need to pass it a ddof of 1 to get the sample standard deviation.

s = np.std(sample, ddof=1)
s

7.496892605369556

In [46]:
# The next step is to find the interval

from scipy.stats import t

In [47]:
# we will use symmetric tails with density 0.005

a = -t.ppf(0.005, n-1)
a

2.860934606449914

In [48]:
# check that our interval has the correct density

t.cdf(a, n-1) - t.cdf(-a, n-1)

0.98999999999967

So what we now have is that:

$$ 0.99 = P(-a < \sqrt{n} \frac{\bar{Y} - \theta}{s} < a ) $$

Solving for $\theta$ we get:

$$ \bar{Y} - \frac{a s}{\sqrt{n}} < \theta < \bar{Y} + \frac{a s}{\sqrt{n}} $$

Or in other words $\theta$ is in the following interval with 99% probability:

In [49]:
(Ybar - a*s/np.sqrt(n), Ybar + a*s/np.sqrt(n) )

(3.5747734479190862, 13.166664084232373)

## Discussion

Note that 99% confident is pretty confident. Redo the computaiton above but looking for an interval with only 90% confidence. 10% of the time we will have the wrong conclusion!

In [55]:
n = 20

for k in range(100):
    sample = expon.rvs(size=n)*10
    Ybar = np.mean(sample)
    s = np.std(sample, ddof=1)
    a = -t.ppf(0.05, n-1)
    
    l = Ybar - a*s/np.sqrt(n)
    u = Ybar + s*s/np.sqrt(n)
    
    # check if 10 is in the interval and print in red; else print in black  
    if l > 10 or u < 10:
        print('\033[91m' + str((l, u )) + '\033[0m' )
    else:
        print((l, u))

(7.477985959698341, 24.879406741880793)
(9.277953916959827, 77.9889381510716)
(4.338372721491339, 13.992406613032035)
(7.463594750054622, 60.31831376424357)
(7.188019956516205, 36.027356722177444)
(8.967238261321958, 53.02527606605766)
(5.902516531682981, 32.63761447475815)
(7.2699523099716075, 25.802979580811957)
(7.495644728782777, 13.012419336388493)
(7.189229074117173, 28.866281413963748)
(5.706330422072678, 30.5612745680189)
(3.9732775170436874, 48.630525941911934)
(6.998418061050199, 37.24616018547762)
(4.286709315057645, 19.353782328583627)
(4.2106487951102185, 11.523515730208675)
(6.326189784960032, 30.438793275631593)
(8.72889882950816, 29.552269169637153)
(6.881272559967278, 37.06364137440315)
(5.609617515601211, 33.113831379729504)
(8.543122038981567, 40.60742687906016)
(4.659622208682716, 25.985087147091107)
(5.60706758750363, 24.12573354671986)
(5.332104600080017, 32.952313598651145)
(5.574947773390649, 44.76389508304808)
(3.9765457192423135, 26.585922972951828)
(5.9932285

## Mean Differences

The following prices of White Tuna Packed in Oil or Water were collected by sampling major brands at a local super market:

In [56]:
oil = [1.27, 1.22, 1.19, 1.22]
water = [1.49, 1.29, 1.27, 1.35, 1.29, 1.00, 1.27, 1.28]

The sample means are:

In [57]:
np.mean(oil), np.mean(water)

(1.225, 1.28)

We wish to find a confidence interval for the difference between the population means for White Tuna Packed in Water or Oil. 

Note that we expect the statistic:

$$ Z = \frac{\bar{Y_1} - \bar{Y_2} - (\mu_1 - \mu_2) }{\sqrt{ \sigma_1^2/n_1 + \sigma_2^2/n_2 }} $$

to be a normally distributed. Assuming that $\sigma$ is equal for the two populations this would be:

$$ Z = \frac{\bar{Y_1} - \bar{Y_2} - (\mu_1 - \mu_2) }{ \sigma \sqrt{ 1/n_1 + 1/n_2} } $$

However note that for this problem we do not know $\sigma_1$ or $\sigma_2$, or $\sigma$ for that matter. The assumption that two very similar products have the same variance is a common one so we could proceed with that and use what is called a **pooled estimator** for the $\sigma$:

$$ S_p^2 = \frac{ (n_1 -1 ) S_1^2 + (n_2 - 1) S_2^2 }{n_1 + n_2 - 2} $$

To explain where this comes from:  

- note that we multiply the individual sample variances by their denominator.
- We compute the sum of the squares of the differences of each sample from the corresponding sample mean and add these
- then we divide by the degrees of freedom:  $n_1 + n_2 - 2$  The reason there are 2 less degrees of freedom is becuase we have two means.

- The other way to think about the pooled estimator is that it is the weighted average of the sample variances; weighted by the number of degrees of freedom.

Then we can use $S_p$ in our Student's T distirbution with $n_1 + n_2 - 2$ degrees of freedom:

$$ T = \frac{ \bar{Y_1} - \bar{Y_2} - (\mu_1 - \mu_2) }{ S_p \sqrt{1/n_1 + 1/n_2} }$$

We will get that:

$$ (\bar{Y}_1 - \bar{Y}_2) \pm t_{\alpha/2} S_p \sqrt{1/n_1 + 1/n_2} $$ 

gives the boundaries of our confidence interval where 

$$ P(-t_{\alpha/2} < T < t_{\alpha/2} ) = 1 - \alpha $$

In [62]:
# Doing the computation:

confidence = 0.99
alpha = 1 - confidence


# Compute the two sample means
Ybar1 = np.mean(water)
Ybar2 = np.mean(oil)

# Record the sample sizes
n1 = len(water)
n2 = len(oil)
dof = n1+n2-2

# Compute the two sample standard deviations
S1 = np.std(water, ddof = 1)
S2 = np.std(oil, ddof = 1)

# Compute the pooled sample standard deviation

Sp = np.sqrt( ((n1-1)*S1 + (n2-2)*S2)/dof )

# Compute the talpha

talpha = -t.ppf(alpha/2, dof)

# Give the confidence interval

( (Ybar1 - Ybar2) - talpha * Sp *np.sqrt(1/n1 + 1/n2), (Ybar1 - Ybar2) + talpha * Sp *np.sqrt(1/n1+1/n2) )


(-0.5624488640252127, 0.6724488640252125)